# Temat testów
### Czy istnieje istotna statystycznie różnica w czasie odpowiedzi w zależności od cech osobowych badanych?

In [16]:
import pandas as pd

from functions.tests.categorical_and_quntitative import conduct_categorical_vs_quantitative_for_many_groups_test
from functions.tests.posthoc import posthocNemenyi_test
from functions.set_preparations import prepare_row
from functions.others import choose_dominant_kind

In [2]:
df_quest = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/survey.csv',
).fillna('NO_VAL')

df_quest = df_quest[df_quest.ID != 6] # odrzucony uczestnik ze względu na metryki eye trackingowe

# zagregowane kolumny - podział na tych co pisali i nie pisali danej matury:
df_quest['MAT'] = df_quest['MAT_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL' )
df_quest['POL'] = df_quest['JP_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL' )
df_quest['HIST'] = df_quest['HIST_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_quest['Typ_uczenia'] = df_quest.apply(choose_dominant_kind, axis=1)
df_quest['Kierunek_studiów'] = df_quest['Kierunek_studiów'].apply(lambda x: 'MISMaP Geografia' if x == 'Geografia, w ramach studiów międzyobszarowych' else x)

df_time = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/times.csv')

df_merge = df_quest.merge(df_time, on='ID')
df_merge.head()

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Duration_2a,Duration_2b,Duration_3a,Duration_3b,SumTime_all,SumTime_a,SumTime_b,SumTime_1,SumTime_2,SumTime_3
0,1,M,2002,MISMaP Geografia,2,SPOŁ-EKO,0,1,6,90 - 100%,...,54481,58697,56314,63897,276921,138810,138110,43531,113178,120211
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,42248,85329,47948,129427,459647,156426,303219,154693,127577,177375
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,45165,32732,8000,61197,299890,121995,177892,152793,77897,69197
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,12449,37715,60980,101795,337437,119477,217956,124494,50164,162775
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,55931,22132,34898,62930,271458,158809,112644,95562,78063,97828


Przygotowanie zbiorów:

In [3]:
# zsumowane wartości
df_sumAll = df_merge.rename(columns={'SumTime_all':'time'})
df_sumA = df_merge.rename(columns={'SumTime_a':'time'})
df_sumB = df_merge.rename(columns={'SumTime_b':'time'})
df_sum1 = df_merge.rename(columns={'SumTime_1':'time'})
df_sum2 = df_merge.rename(columns={'SumTime_2':'time'})
df_sum3 = df_merge.rename(columns={'SumTime_3':'time'})

# kolekcje
df_collAll = pd.concat([df_quest] * 6, ignore_index=True)
df_collAll["time"] = prepare_row(df_time.filter(regex='Duration'))

df_collA = pd.concat([df_quest] * 3, ignore_index=True)
df_collA["time"] = prepare_row(df_time.filter(regex='Duration.*a'))

df_collB = pd.concat([df_quest] * 3, ignore_index=True)
df_collB["time"] = prepare_row(df_time.filter(regex='Duration.*b'))

df_coll1 = pd.concat([df_quest] * 2, ignore_index=True)
df_coll1["time"] = prepare_row(df_time.filter(regex='Duration_1'))

df_coll2 = pd.concat([df_quest] * 2, ignore_index=True)
df_coll2["time"] = prepare_row(df_time.filter(regex='Duration_2'))

df_coll3 = pd.concat([df_quest] * 2, ignore_index=True)
df_coll3["time"] = prepare_row(df_time.filter(regex='Duration_3'))

In [4]:
dfs = [df_sumAll, df_sumA, df_sumB, df_sum1, df_sum2, df_sum3, df_collAll, df_collA, df_collB, df_coll1, df_coll2, df_coll3]
names = {
    1: 'Zsumowane wszystkie grafiki:',
    2: 'Zsumowane grafiki A:',
    3: 'Zsumowane grafiki B:',
    4: 'Zsumowane grafiki 1:',
    5: 'Zsumowane grafiki 2:',
    6: 'Zsumowane grafiki 3:',
    7: 'Kolekcja wszystkie grafiki:',
    8: 'Kolekcja grafiki A:',
    9: 'Kolekcja grafiki B:',
    10: 'Kolekcja grafiki 1:',
    11: 'Kolekcja grafiki 2:',
    12: 'Kolekcja grafiki 3:',
}

columns = ['Płeć', 'Rok_urodzenia', 'Kierunek_studiów',
       'Rok_studiów', 'Specjalność', 'Wady_wzroku', 'Ilość_snu',
       'JP_podst', 'MAT_podst', 'JP_roz', 'MAT_roz', 'HIST_roz',
       'Samopocz', 'Stres', 'Zmęcz', 'Czyt_map', 'Podziel_uwg', 'Analit_myśl',
       'Spostrzeg', 'Czyt_ze_zroz', 'Wzrokowiec', 'Słuchowiec', 'Kinestetyk', 'MAT', 'POL', 'HIST', 'Typ_uczenia']

**H0:** Nie ma istotnej statystycznie zależności między czasem odpowiedzi a wartością cechy X.

**H1:** Istnieje istotna statystycznie zależność między czasem odpowiedzi a wartością cechy X.

**Poziom istotności:** $\alpha=0.05$

In [12]:
ALPHA = 0.05

In [18]:
for col in columns:
    print('################################################')
    print(col)
    for i, df in enumerate(dfs):
        print('-------------------------')
        print(names[i+1])
        conduct_categorical_vs_quantitative_for_many_groups_test(df, 'time', col, ALPHA)

################################################
Płeć
-------------------------
Zsumowane wszystkie grafiki:
Conducting normality test...
Kruskal-Wallis test conducted...
H: 0.3460170918844767
p-value: 0.5563764636405055
Brak podstaw do odrzucenia H0.
-------------------------
Zsumowane grafiki A:
Conducting normality test...
Conducting homogeneity variance test...
Kruskal-Wallis test conducted...
H: 0.03162622419064576
p-value: 0.8588504904838072
Brak podstaw do odrzucenia H0.
-------------------------
Zsumowane grafiki B:
Conducting normality test...
Conducting homogeneity variance test...
Kruskal-Wallis test conducted...
H: 0.7906556047657745
p-value: 0.373901958627471
Brak podstaw do odrzucenia H0.
-------------------------
Zsumowane grafiki 1:
Conducting normality test...
Conducting homogeneity variance test...
Kruskal-Wallis test conducted...
H: 1.2278086207972052
p-value: 0.267833632802634
Brak podstaw do odrzucenia H0.
-------------------------
Zsumowane grafiki 2:
Conducting n

Podsumowanie:
Istotne statystycznie:
- Rok urodzenia: zsumowane grafiki 2 (słabo się różnicuje), kolekcja wszystkie grafiki (2002 vs 2000, 2003, 2004 oraz 2003 vs reszta), 
- Rok studiów: kolekcja:  wszystkie grafiki, grafiki B (w obu przypadkach 1 rok vs pozostałe)
- Matura: j.polski podstawa: zsumowane grafiki 1 (słabo się różnicuje)
- Matura: matematyka podstawa: kolekcja: wszystkie grafiki (wyższe wyniki z niższymi wynikami), grafiki A (90-100 ze wszystkimi), grafiki 3 (90-100 ze wszystkimi poza 30-50)
- Matura: j.polski rozszerzony: zsumowane: wszystkie grafiki, grafiki B, grafiki 1, grafiki 3, kolekcje: wszystkie grafiki, B (generalnie wszystkie grupy vs ci co nie pisali)
- Samopoczucie: zsumowane grafiki: wszystkie, B, 2, 3 (2 vs 4)
- Zmęczenie: kolekcja grafiki: wszystkie (5 vs pozostałe, 1 vs 2)
- Czytanie ze zrouzumieniem: zsumowane grafiki: 1 (słabo się różnicuje)
- Kinestetyk: kolekcje grafik: wszystkie, B (praktycznie wszystkie)

In [6]:
# posthoc tests
posthocNemenyi_test(df_sum2, 'time', 'Rok_urodzenia')
print()
posthocNemenyi_test(df_collAll, 'time', 'Rok_urodzenia')
print()
posthocNemenyi_test(df_collAll, 'time', 'Rok_studiów')
print()
posthocNemenyi_test(df_collB, 'time', 'Rok_studiów')

+------+----------+----------+----------+----------+----------+
|      |     2002 |     2003 |     2001 |     2000 |     2004 |
+======+==========+==========+==========+==========+==========+
| 2002 | 1        | 0.764018 | 0.960921 | 0.73919  | 0.584269 |
+------+----------+----------+----------+----------+----------+
| 2003 | 0.764018 | 1        | 0.364138 | 0.439093 | 0.151043 |
+------+----------+----------+----------+----------+----------+
| 2001 | 0.960921 | 0.364138 | 1        | 0.896714 | 0.882736 |
+------+----------+----------+----------+----------+----------+
| 2000 | 0.73919  | 0.439093 | 0.896714 | 1        | 0.998702 |
+------+----------+----------+----------+----------+----------+
| 2004 | 0.584269 | 0.151043 | 0.882736 | 0.998702 | 1        |
+------+----------+----------+----------+----------+----------+

+------+----------+----------+----------+----------+----------+
|      |     2002 |     2003 |     2001 |     2000 |     2004 |
+======+==========+==========+=========

In [7]:
posthocNemenyi_test(df_sum1, 'time', 'JP_podst')
print()
posthocNemenyi_test(df_collAll, 'time', 'MAT_podst')
print()
posthocNemenyi_test(df_collA, 'time', 'MAT_podst')
print()
posthocNemenyi_test(df_coll3, 'time', 'MAT_podst')

+-----------+-------------+------------+------------+------------+
|           |   90 - 100% |   50 - 69% |   70 - 89% |   30 - 49% |
+===========+=============+============+============+============+
| 90 - 100% |    1        |   0.353282 |    0.09182 |   1        |
+-----------+-------------+------------+------------+------------+
| 50 - 69%  |    0.353282 |   1        |    0.83523 |   0.353282 |
+-----------+-------------+------------+------------+------------+
| 70 - 89%  |    0.09182  |   0.83523  |    1       |   0.09182  |
+-----------+-------------+------------+------------+------------+
| 30 - 49%  |    1        |   0.353282 |    0.09182 |   1        |
+-----------+-------------+------------+------------+------------+

+-----------+-------------+------------+------------+------------+
|           |   90 - 100% |   70 - 89% |   50 - 69% |   30 - 49% |
+===========+=============+============+============+============+
| 90 - 100% |     1       |   1e-06    |   0        |   5.5e-

In [8]:
posthocNemenyi_test(df_sumAll, 'time', 'JP_roz')
print()
posthocNemenyi_test(df_sumB, 'time', 'JP_roz')
print()
posthocNemenyi_test(df_sum1, 'time', 'JP_roz')
print()
posthocNemenyi_test(df_sum3, 'time', 'JP_roz')
print()
posthocNemenyi_test(df_collAll, 'time', 'JP_roz')
print()
posthocNemenyi_test(df_collB, 'time', 'JP_roz')

+-------------+---------------+------------+----------+------------+
|             |   powyżej 70% |   50 - 70% |   NO_VAL |   20 - 49% |
+=============+===============+============+==========+============+
| powyżej 70% |      1        |   0.992107 | 0.073875 |   0.996057 |
+-------------+---------------+------------+----------+------------+
| 50 - 70%    |      0.992107 |   1        | 0.045699 |   0.97431  |
+-------------+---------------+------------+----------+------------+
| NO_VAL      |      0.073875 |   0.045699 | 1        |   0.331673 |
+-------------+---------------+------------+----------+------------+
| 20 - 49%    |      0.996057 |   0.97431  | 0.331673 |   1        |
+-------------+---------------+------------+----------+------------+

+-------------+---------------+------------+----------+------------+
|             |   powyżej 70% |   50 - 70% |   NO_VAL |   20 - 49% |
+=============+===============+============+==========+============+
| powyżej 70% |      1        |  

In [9]:
posthocNemenyi_test(df_sumAll, 'time', 'Samopocz')
print()
posthocNemenyi_test(df_sumB, 'time', 'Samopocz')
print()
posthocNemenyi_test(df_sum2, 'time', 'Samopocz')
print()
posthocNemenyi_test(df_sum3, 'time', 'Samopocz')

+----+----------+----------+----------+----------+
|    |        4 |        3 |        5 |        2 |
+====+==========+==========+==========+==========+
|  4 | 1        | 0.066631 | 0.219529 | 0.04679  |
+----+----------+----------+----------+----------+
|  3 | 0.066631 | 1        | 0.90017  | 0.994794 |
+----+----------+----------+----------+----------+
|  5 | 0.219529 | 0.90017  | 1        | 0.793153 |
+----+----------+----------+----------+----------+
|  2 | 0.04679  | 0.994794 | 0.793153 | 1        |
+----+----------+----------+----------+----------+

+----+----------+----------+----------+----------+
|    |        4 |        3 |        5 |        2 |
+====+==========+==========+==========+==========+
|  4 | 1        | 0.066631 | 0.219529 | 0.04679  |
+----+----------+----------+----------+----------+
|  3 | 0.066631 | 1        | 0.90017  | 0.994794 |
+----+----------+----------+----------+----------+
|  5 | 0.219529 | 0.90017  | 1        | 0.793153 |
+----+----------+----------+--

In [10]:
posthocNemenyi_test(df_collAll, 'time', 'Zmęcz')
print()
posthocNemenyi_test(df_sum1, 'time', 'Czyt_ze_zroz')
print()
posthocNemenyi_test(df_collAll, 'time', 'Kinestetyk')
print()
posthocNemenyi_test(df_collB, 'time', 'Kinestetyk')

+----+----------+----------+----------+----------+----------+
|    |        3 |        2 |        4 |        1 |        5 |
+====+==========+==========+==========+==========+==========+
|  3 | 1        | 0.729747 | 0.961537 | 0.053367 | 0.000338 |
+----+----------+----------+----------+----------+----------+
|  2 | 0.729747 | 1        | 0.303819 | 0.000736 | 3e-06    |
+----+----------+----------+----------+----------+----------+
|  4 | 0.961537 | 0.303819 | 1        | 0.254589 | 0.00336  |
+----+----------+----------+----------+----------+----------+
|  1 | 0.053367 | 0.000736 | 0.254589 | 1        | 0.394665 |
+----+----------+----------+----------+----------+----------+
|  5 | 0.000338 | 3e-06    | 0.00336  | 0.394665 | 1        |
+----+----------+----------+----------+----------+----------+

+----+----------+----------+----------+----------+----------+
|    |        4 |        2 |        3 |        1 |        5 |
+====+==========+==========+==========+==========+==========+
|  4 | 

Dla każdej grafiki oddzielnie:

In [13]:
columns = ['Płeć', 'Rok_urodzenia', 'Kierunek_studiów',
           'Rok_studiów', 'Specjalność', 'Wady_wzroku', 'Ilość_snu',
           'JP_podst', 'MAT_podst', 'JP_roz', 'MAT_roz', 'HIST_roz',
           'Samopocz', 'Stres', 'Zmęcz', 'Czyt_map', 'Podziel_uwg', 'Analit_myśl',
           'Spostrzeg', 'Czyt_ze_zroz', 'Wzrokowiec', 'Słuchowiec', 'Kinestetyk', 'MAT', 'POL', 'HIST', 'Typ_uczenia']

prefix = ['1a', '1b', '2a', '2b', '3a', '3b']
for col in columns:
    print('##############################################################')
    print(col)
    for pref in prefix:
        print('-------------------------------------')
        print(pref)
        conduct_categorical_vs_quantitative_for_many_groups_test(df_merge, f'Duration_{pref}', col, ALPHA)

##############################################################
Płeć
-------------------------------------
1a
Conducting normality test...
Kruskal-Wallis test conducted...
H: 0.009169733641073208
p-value: 0.9237121983917344
Brak podstaw do odrzucenia H0.
-------------------------------------
1b
Conducting normality test...
Kruskal-Wallis test conducted...
H: 1.688915226748179
p-value: 0.19374402789332795
Brak podstaw do odrzucenia H0.
-------------------------------------
2a
Conducting normality test...
Kruskal-Wallis test conducted...
H: 0.2846360177156697
p-value: 0.5936785815636907
Brak podstaw do odrzucenia H0.
-------------------------------------
2b
Conducting normality test...
Conducting homogeneity variance test...
Kruskal-Wallis test conducted...
H: 0.25619112968622915
p-value: 0.6127490590629245
Brak podstaw do odrzucenia H0.
-------------------------------------
3a
Conducting normality test...
Conducting homogeneity variance test...
Kruskal-Wallis test conducted...
H: 0.13642

In [14]:
posthocNemenyi_test(df_merge, 'Duration_2b', 'Samopocz')
print()
posthocNemenyi_test(df_merge, 'Duration_3a', 'Samopocz')
print()
posthocNemenyi_test(df_merge, 'Duration_1b', 'JP_roz')
print()
posthocNemenyi_test(df_merge, 'Duration_3b', 'JP_roz')
print()
posthocNemenyi_test(df_merge, 'Duration_1b', 'JP_podst')

+----+----------+----------+----------+----------+
|    |        4 |        3 |        5 |        2 |
+====+==========+==========+==========+==========+
|  4 | 1        | 0.066631 | 0.219529 | 0.04679  |
+----+----------+----------+----------+----------+
|  3 | 0.066631 | 1        | 0.90017  | 0.994794 |
+----+----------+----------+----------+----------+
|  5 | 0.219529 | 0.90017  | 1        | 0.793153 |
+----+----------+----------+----------+----------+
|  2 | 0.04679  | 0.994794 | 0.793153 | 1        |
+----+----------+----------+----------+----------+

+----+----------+----------+----------+----------+
|    |        4 |        3 |        5 |        2 |
+====+==========+==========+==========+==========+
|  4 | 1        | 0.066631 | 0.219529 | 0.04679  |
+----+----------+----------+----------+----------+
|  3 | 0.066631 | 1        | 0.90017  | 0.994794 |
+----+----------+----------+----------+----------+
|  5 | 0.219529 | 0.90017  | 1        | 0.793153 |
+----+----------+----------+--

Podsumowanie - dla każdej grafiki:

Istotne statystycznie:
- Samopoczucie: 2b, 3a (2 vs 4)
- JP_roz: 1b, 3b (50-70 vs nie pisał)
- JP_podst: 1b (słabo się różnicuje)